In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data Loading 

In [ ]:
sales_df = pd.read_csv('../../datasets/predict-sales/sales_train.csv')
items_df = pd.read_csv('../../datasets/predict-sales/items.csv')
test_df = pd.read_csv('../../datasets/predict-sales/test.csv')

In [ ]:
sales_df.drop(labels=['date'],inplace=True,axis=1)
sales_df = sales_df.reset_index()
items_df.drop(labels=['item_name'],inplace=True,axis=1)

In [ ]:
sales_df.info()

In [ ]:
sales_df.head()

# Visualizing Data 

Let's visualize some data from the test set:

In [ ]:
def drawTimeSerie(df, sample, n):
    for i in range(n):
        shop_id = sample.iloc[i]['shop_id']
        item_id = sample.iloc[i]['item_id']
        id_df = df[(df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
        id_df_grouped = id_df[['date_block_num','item_cnt_day']].groupby('date_block_num').sum().reset_index()
        plt.figure(figsize=[10,n*5])
        plt.subplot(n,1,i+1)
        plt.plot(id_df_grouped['date_block_num'], id_df_grouped['item_cnt_day'],'*-',)
        plt.title(str(shop_id) + '-' + str(item_id))

In [ ]:
n_samples = 10
sample = test_df.sample(n=n_samples, axis=0)

In [ ]:
drawTimeSerie(sales_df, sample, n_samples)

# Structure of Data

In [ ]:
import featuretools as ft

Let's evaluate two kinds of process 

## Pre-proccesing

In [ ]:
month_id = 15

In [ ]:
sales_month_df = sales_df[sales_df['date_block_num'] == month_id]

In [ ]:
sales_month_df['ID_pair'] = sales_month_df[['shop_id','item_id']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

In [ ]:
sales_month_df.drop(labels=['shop_id'], inplace=True, axis=1)

In [ ]:
sales_month_df.head()

## EntitySet Processing (Recommended by FeatureTools)

In [ ]:
es = ft.EntitySet(id="prediction_sales")

In [ ]:
es = es.entity_from_dataframe(entity_id='sales',dataframe=sales_month_df, index='index')

In [ ]:
es = es.entity_from_dataframe(entity_id='items',dataframe=items_df, index='item_id',variable_types={'item_category_id': ft.variable_types.Categorical})

In [ ]:
new_relationship = ft.Relationship(es['items']["item_id"], es['sales']['item_id'])

In [ ]:
es = es.add_relationship(new_relationship)

In [ ]:
es = es.normalize_entity(base_entity_id='sales',
                         new_entity_id='ids',
                         index='ID_pair'
                        )

In [ ]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity='ids')

In [ ]:
feature_matrix[feature_matrix['COUNT(sales)'] > 1].head()

In [ ]:
sns.distplot(feature_matrix['COUNT(sales)'])

# End of Case! 